# **Requête API avec OpenFoodFacts**

## Introduction

🎯 **Objectif**: tester la collecte de données de produits alimentaires contenant du champagne à l’aide de l'API OpenFoodFacts (L'API OpenFoodFacts est une API **libre**, **collaborative** et **gratuite** qui permet d’accéder à des informations nutritionnelles et commerciales sur des milliers de produits alimentaires à travers le monde).

- Rechercher des produits liés à l’épicerie fine à base de champagne

- Extraire les 10 premiers produits et enregistrer les informations suivantes dans un fichier .csv :

    - foodId : identifiant unique du produit
    - label : nom du produit
    - category : catégorie du produit
    - foodContentsLabel : ingrédients
    - image : lien vers l’image du produit


🔎 **Méthodologie**

Le notebook est structuré en plusieurs étapes :

1. Requête et traitement des résultats
2. Export des données au format .csv
3. Visualisations


🔗**Endpoint utilisé**: 

https://world.openfoodfacts.org/api/v2/search


✅ **Paramètres utilisés de l'API**:

- *ingredients_tags* : filtre sur les ingrédients (ici, champagne)
- *page_size* : limite le nombre de produits retournés
- *fields* : permet de sélectionner les champs qu’on veut dans la réponse JSON (code, nom, image, etcatgéorie)

In [14]:
import pandas as pd
import requests #librairie pour requête API

import matplotlib.pyplot as plt
from IPython.display import Image, display

In [16]:
# 1. Fonction pour accéder à l'API et la requêter

def get_openfoodfacts_products(query="champagne", max_results=10):
    # Lien de l’endpoint de recherche de produits (API OpenFoodFacts)
    url = "https://world.openfoodfacts.org/api/v2/search"

    # Paramètres qu’on ajoute à l’URL pour filtrer la recherche
    params = {
        "ingredients_tags": query,  # cherche seulement les produits contenant champagne
        "page_size": max_results,            # limite à 10 résultats
        "fields": "code,product_name,categories,ingredients_text,image_url" # sort seulement les colonnes demandées
    }

    # Requête GET vers l’API avec les paramètres
    response = requests.get(url, params=params)

    # Transforme la réponse en dictionnaire Python (format JSON)
    data = response.json()

    # Liste des résultats
    results = []

    # Boucle sur chaque produit dans les résultats retournés
    for product in data.get("products", []):
        results.append({
            "foodId": product.get("code"),                        # ID du produits
            "label": product.get("product_name"),                 # Nom du produit
            "category": product.get("categories"),                # Catégories du produit
            "foodContentsLabel": product.get("ingredients_text"), # Ingrédients du produit
            "image": product.get("image_url")                     # URL image
        })

    return pd.DataFrame(results)


In [18]:
# 2. Export de la dataframe

# Appel de la fonction
df_openfoodfacts = get_openfoodfacts_products()

# Sauvegarde en document csv
df_openfoodfacts.to_csv("produits_openfoodfacts_champagne.csv", index=False)

# Affichage de la df
display(df_openfoodfacts)

,foodId,label,category,foodContentsLabel,image
0,8711812380571,Faux Gras,"Produits à tartiner, Produits à tartiner salés...","Eau, levure alimentaire, huile de coco, amidon...",https://images.openfoodfacts.org/images/produc...
1,4005514008807,Wouah gras vegan,"Meat alternatives,fr:Faux Gras,fr:Plat-festif,...","Eau, levure alimentaire*, huile de coco*, amid...",https://images.openfoodfacts.org/images/produc...
2,4056489843696,Rillettes de homard au cognac,"Seafood, Fishes and their products, Fish prepa...","Chair de homard américain 49%, huile de colza,...",https://images.openfoodfacts.org/images/produc...
3,3258431220000,,"Boissons, Boissons alcoolisées, Vins, Vins eff...",Champagne,https://images.openfoodfacts.org/images/produc...
4,3049610004104,Veuve Clicquot Champagne Ponsardin Brut,"Boissons et préparations de boissons, Boissons...",Champagne,https://images.openfoodfacts.org/images/produc...
5,3282946015837,Nicolas Feuillatte,"Boissons, Boissons alcoolisées, Vins, Vins fra...","Champagne, Contient des _sulfites_",https://images.openfoodfacts.org/images/produc...
6,3416181017169,"Champagne AOP, brut","Boissons, Boissons alcoolisées, Vins, Vins eff...",Champagne,https://images.openfoodfacts.org/images/produc...
7,3185370283905,Champagne Ruinart,"Boissons, Boissons alcoolisées, Vins, Vins eff...",champagne,https://images.openfoodfacts.org/images/produc...
8,3245391237858,Champagne CHARLES VINCENT BRUT,"Boissons, Boissons alcoolisées, Vins, Vins fra...",Champagne brut.,https://images.openfoodfacts.org/images/produc...
9,3256930103817,Champagne Blue Top Brut,"Boissons et préparations de boissons, Boissons...",Champagne (_sulfites_),https://images.openfoodfacts.org/images/produc...


In [20]:
# S'assurer que dans les 3 premiers produits il y a bien du champagne dans les ingrédients: 

print("📋 Liste complète des ingrédients pour les 3 premiers produits :\n")

for idx, row in df_openfoodfacts.head(3).iterrows():
    print(f"Produit {idx+1} - {row['label']}")
    print(f"Ingrédients : {row['foodContentsLabel']}\n")
    print("-" * 80)

📋 Liste complète des ingrédients pour les 3 premiers produits :

Produit 1 - Faux Gras
Ingrédients : Eau, levure alimentaire, huile de coco, amidon de pomme de terre, protéines de tournesol, Champagne (2%), pulpe de tomates, sel marin, huile de tournesol, truffe (1%), épices (0,18% : coriandre, cannelle, clous de girofle).

--------------------------------------------------------------------------------
Produit 2 - Wouah gras vegan
Ingrédients : Eau, levure alimentaire*, huile de coco*, amidon de pomme de terre*, protéines végétales de graines de tournesol*, Champagne* 2%, double concentré de tomates*, sel marin, huile de tournesol*, truffes* déshydratées 1%, épices* (coriandre*, cannelle*, clou de girofle*)
* issus de l'agriculture biologique

--------------------------------------------------------------------------------
Produit 3 - Rillettes de homard au cognac
Ingrédients : Chair de homard américain 49%, huile de colza, colin d'Alaska, eau, double concentré de tomates, Champagne (

In [22]:
# ==== 3. Visualisations ====

for idx, row in df_openfoodfacts.iterrows():
    print(f"🧾 Produit : {row['label']}")
    print(f"📂 Catégorie : {row['category']}")
    if pd.notna(row['image']):
        display(Image(url=row['image'], width=200))
    print("-" * 50)

🧾 Produit : Faux Gras
📂 Catégorie : Produits à tartiner, Produits à tartiner salés, Alternatives à la viande, Substituts de viande, Substituts du foie gras


--------------------------------------------------
🧾 Produit : Wouah gras vegan
📂 Catégorie : Meat alternatives,fr:Faux Gras,fr:Plat-festif,Foie gras substitutes


--------------------------------------------------
🧾 Produit : Rillettes de homard au cognac
📂 Catégorie : Seafood, Fishes and their products, Fish preparations, fr:Rillettes de homard au cognac et champagne, en:lobster-rillettes


--------------------------------------------------
🧾 Produit : 
📂 Catégorie : Boissons, Boissons alcoolisées, Vins, Vins effervescents, Champagnes, Champagnes bruts


--------------------------------------------------
🧾 Produit : Veuve Clicquot Champagne Ponsardin Brut
📂 Catégorie : Boissons et préparations de boissons, Boissons, Boissons alcoolisées, Vins, Vins effervescents, Champagnes


...

--------------------------------------------------
🧾 Produit : Champagne AOP, brut
📂 Catégorie : Boissons, Boissons alcoolisées, Vins, Vins effervescents, Champagnes, Champagnes bruts


--------------------------------------------------
🧾 Produit : Champagne Ruinart
📂 Catégorie : Boissons, Boissons alcoolisées, Vins, Vins effervescents, Champagnes


--------------------------------------------------
🧾 Produit : Champagne CHARLES VINCENT BRUT
📂 Catégorie : Boissons, Boissons alcoolisées, Vins, Vins français, Vins effervescents, Champagnes, Champagnes français


--------------------------------------------------
🧾 Produit : Champagne Blue Top Brut
📂 Catégorie : Boissons et préparations de boissons, Boissons, Boissons alcoolisées, Vins, Vins français, Vins effervescents, Champagnes, Champagnes français, Champagnes bruts


--------------------------------------------------
